In [3]:
from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras import regularizers
from keras.models import Model
from keras import optimizers
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from cv2 import cv2
from google.colab import drive
import pandas as pd 
import keras 
from keras.models import load_model
import os

Using TensorFlow backend.


In [4]:
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/APM'  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
visible = Input(shape=(256,256,1))
conv1 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(visible)
conv2 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(conv1)
bat1 = BatchNormalization()(conv2)
zero1 = ZeroPadding2D(padding=(1, 1))(bat1)

conv3 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(zero1)
conv4 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(conv3)
bat2 = BatchNormalization()(conv4)

conv5 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(bat2)
conv6 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(conv5)
bat3 = BatchNormalization()(conv6)
pool1 = MaxPooling2D(pool_size=(2, 2))(bat3)
zero2 = ZeroPadding2D(padding=(1, 1))(pool1)

conv7 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.01))(zero2)
conv8 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv7)
bat4 = BatchNormalization()(conv8)

conv15 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(bat4)
conv16 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv8)
bat8 = BatchNormalization()(conv16)

flat = Flatten()(bat8)
hidden1 = Dense(32, activation='relu')(flat)
drop1 = Dropout(0.3)(hidden1)

hidden2 = Dense(32, activation='relu')(drop1)
drop2 = Dropout(0.2)(hidden2)

output = Dense(8, activation='sigmoid')(drop2)
model = Model(inputs=visible, outputs=output)

opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer= opt, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 254, 254, 16)      160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 16)      2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 252, 252, 16)      64        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 254, 254, 16)      0         
__________________________________

In [3]:
cd '/content/gdrive/My Drive/APM/Dataset/unzipped/'

/content/gdrive/My Drive/APM/Dataset/unzipped


In [0]:
test_on = ['Actor_17', 'Actor_20']
def video_generator(skip):
  j = 1
  X=[]
  Y=[]
 
  for actor in os.listdir('./'):
  #for actor in [('Actor_01'),]:
    print(actor)

    if ('Actor' not in actor) or (actor in skip):
      
      continue # skipping the extra folders
    path = os.path.join('./',actor)
    
    # progress tracking
    l = len(os.listdir(path))
    i = 0
    
    #print('Training on',actor)
    j += 1

    for filename in os.listdir(path):
      
      print(filename)
      path_inner = os.path.join(path, filename)
      file_emotion = [0]*8
      file_emotion[int(path_inner[17:19])-1] = 1
      print(file_emotion)
      print(path_inner)

      cap = cv2.VideoCapture(path_inner)
      x = []
      y = []
      
     

      frame_count = 0
      while True:
        
        frame_count += 1
        # read the next frame from the file
        (grabbed, frame) = cap.read()

        # If the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
          
          #print('Done,',i,'frames captured.')
          break
        # only grab each 5th frame to capture a slightly different expression each time; makes computation 5x faster
        elif frame_count % 20 == 0:
          frame= cv2.resize(frame, (256,256))
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          frame = np.reshape(frame, [frame.shape[0], frame.shape[1], 1])
          x.append(frame)
          y.append(file_emotion)

      
      i += 1
      #print('    Training on',filename, int(i/l*100), '%')
      #cnn.train_on_batch(x,y)
      x = np.array(x)
      y = np.array(y)
      i += 1
      #print('    Training on',filename, int(i/l*100), '%')
      #cnn.train_on_batch(x,y)
      yield (x,y)
      del x
      del y
      
      
  

In [7]:
model.fit_generator(video_generator(test_on), steps_per_epoch=25,epochs = 15,verbose = 1)
model.save('Video_Emotion_Detection_25.hdf5')

Running

In [7]:
cd '/content/gdrive/My Drive/APM/Dataset/unzipped'

/content/gdrive/My Drive/APM/Dataset/unzipped


In [9]:
model=load_model('09.hdf5')













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [38]:
cd '/content/gdrive/My Drive/APM/Dataset/unzipped/'

/content/gdrive/My Drive/APM/Dataset/unzipped


In [0]:
def testOnActor(actor):
  actor_dir = './'+actor+'/'
  loss = []
  acc = []
  #Test
  i = 0
  for filename in os.listdir(actor_dir):
    i += 1
    #if i > 10:
    #  break
    path = os.path.join(actor_dir, filename)
    file_emotion = [0]*8
    file_emotion[int(path[17:19])-1] = 1

    cap = cv2.VideoCapture(path)
    x = []
    y = []
    frame_count = 0
    while True:
      # read the next frame from the file
      (grabbed, frame) = cap.read()
      frame_count += 1

      # If the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
        #print('Done,',i,'frames captured.')
        break
      
      elif frame_count % 10 == 0:
      
        frame=cv2.resize(frame, (256,256))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = np.reshape(frame, [frame.shape[0], frame.shape[1], 1])

        x.append(frame)
        y.append(file_emotion)
        

    x = np.array(x)
    y = np.array(y)
    
    
    #print('Testing on',filename)
    test = model.test_on_batch(x,y)

    

    loss.append(test[0])
    acc.append(test[1])
    del x
    del y
    

  return loss,acc




In [13]:
loss23,acc23 = testOnActor('Actor_23')
print('Actor 23:')
print('Average Loss =',np.array(loss23).mean())
print('Average Accuracy =',np.array(acc23).mean())

Actor 23:
Average Loss = 6.4153695
Average Accuracy = 0.6


In [14]:
loss24,acc24 = testOnActor('Actor_24')
print('Actor 24:')
print('Average Loss =',np.array(loss24).mean())
print('Average Accuracy =',np.array(acc24).mean())


Actor 24:
Average Loss = 6.4153695
Average Accuracy = 0.6
